# Multi Level Market-Basket Analysis

# 1. Data Preprocessing

In [2]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib as mlp

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [17]:
print(np.__version__)
print(pd.__version__)
print(mlp.__version__)

1.23.5
1.5.3
3.6.3


In [3]:
df = pd.read_excel('/kaggle/input/market-basket-analysis/Assignment-1_Data.xlsx')
df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      522064 non-null  object        
 1   Itemname    520609 non-null  object        
 2   Quantity    522064 non-null  int64         
 3   Date        522064 non-null  datetime64[ns]
 4   Price       522064 non-null  float64       
 5   CustomerID  388023 non-null  float64       
 6   Country     522064 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 27.9+ MB


Dalam dataset ini, setiap entry mewakili suatu item yang dibeli dalam suatu Bill.

**Attribut:**

**BillNo:** Angka 6-digit yang ditetapkan untuk setiap transaksi sebagai identifier. 

**ItemName:** Nama produk yang dibeli dalam suatu transaksi. 

**Quantity:** Kuantitas setiap produk yang dibeli dalam suatu transaksi.  

**Date:** Hari dan waktu dari suatu transaksi.  

**Price:** Harga setiap produk 9(satu unit item). 

**CustomerID:** Setiap pelanggan diberi nomor 5 digit sebagai pengenal unik mereka. 

**Country:** Nama negara tempat setiap pelanggan tinggal. 

## 1.1 Mencari data yang tidak valid

### 1.1.1 ItemName

In [6]:
df[df['Itemname'].isna()]

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
613,536414,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1937,536545,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1938,536546,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1939,536547,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1940,536549,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...
515623,581199,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
515627,581203,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
515633,581209,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
517266,581234,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


Terlihat 1455 entries dimana Item name kosong, karena ini MBA, Itemname merupakan suatu feature krusial yang harus ada, jadi data yang tidak punya Itemname bisa dianggap tidak signifikan dan boleh dibuang. 

In [4]:
# Menghapus data dengan missing value itemname
df = df[df['Itemname'].notna()]

### 1.1.2 Quantity

In [9]:
df[df['Quantity']<0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473 entries, 7122 to 519172
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   BillNo      473 non-null    object        
 1   Itemname    473 non-null    object        
 2   Quantity    473 non-null    int64         
 3   Date        473 non-null    datetime64[ns]
 4   Price       473 non-null    float64       
 5   CustomerID  0 non-null      float64       
 6   Country     473 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 29.6+ KB


Diatas kita bisa melihat ada 473 bill entries yang quantity nya dibawah 0. Sebelum rows nya di-drop lebih baik jika kita melakukan eksplorasi data untuk mencari alasan terhadap quantity negative ini dan potensinya terhadap analisis.

In [5]:
temp = df[df['Quantity']<0]
body = temp['Itemname'].to_string(index=False)
names = [item.strip() for item in body.split('\n') if item.strip()]
item_counts = Counter(names)
pd.DataFrame(list(item_counts.items()), columns=['Itemname', 'Count'], index=None)

,Itemname,Count
0,?,41
1,check,120
2,damages,45
3,faulty,1
4,Dotcom sales,2
...,...,...
132,wet,1
133,wet boxes,1
134,????damages????,1
135,mixed up,2


Kita dapat melihat bahwa ternyata barang yang rusak atau tidak dapat dijual ditandai dengan Kuantitas negatif. Entry-entry ini tidak akan berdampak dengan analisis MBA karena tidak terjual kepada suatu customer. Dapat disimpulkan entries dengan kuantitas negatif dapat dibuang.

In [6]:
# Drop entries dengan quantity negatif
df = df.loc[df['Quantity'] > 0]

### 1.1.3 Prices

In [23]:
df.loc[df['Price'] <= 0]

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
6275,536941,amazon,20,2010-12-03 12:08:00,0.0,NaN,United Kingdom
6276,536942,amazon,15,2010-12-03 12:08:00,0.0,NaN,United Kingdom
9097,537197,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
14038,537534,CREAM SWEETHEART LETTER RACK,1,2010-12-07 11:48:00,0.0,NaN,United Kingdom
14039,537534,ZINC WILLIE WINKIE CANDLE STICK,1,2010-12-07 11:48:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...
505251,580609,Amazon,1,2011-12-05 11:41:00,0.0,NaN,United Kingdom
515626,581202,check,41,2011-12-07 18:30:00,0.0,NaN,United Kingdom
515635,581211,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom
518770,581406,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom


Kita menemukan 585 entries dimana harga barang 0 atau negatif. Kebanyakan entries tidak memiliki customer id dan beberapa entries bukan merupakan nama barang. Dapat disimpilkan bahwa entries ini dapat memberikan informasi inakurat dalam analisis kita sehingga boleh untuk dihilangkan.

In [7]:
# Remove rows price 0
df = df[df['Price'] != 0]

### 1.1.4 Country

In [6]:
print("Number of unique countries:", df['Country'].nunique())
print(df['Country'].value_counts(normalize=True)[:5])

Number of unique countries: 30
United Kingdom    0.933735
Germany           0.017400
France            0.016181
Spain             0.004781
Netherlands       0.004540
Name: Country, dtype: float64


Dominasi transaksi (sekitar 93%) dalam kumpulan data yang berasal dari Inggris Raya (UK), kolom "Country" mungkin tidak memberikan kontribusi signifikan terhadap keragaman atau variabilitas untuk analisis. 

### 1.1.5 CustomerID

In [7]:
df[df['CustomerID'].isna()].sample(30)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
418962,574076,SET OF 4 KNICK KNACK TINS DOILY,1,2011-11-02 15:38:00,8.29,NaN,United Kingdom
33864,539434,RED HANGING HEART T-LIGHT HOLDER,6,2010-12-17 14:41:00,5.91,NaN,United Kingdom
442209,575875,EGG CUP HENRIETTA HEN PINK,1,2011-11-11 13:06:00,2.46,NaN,United Kingdom
483224,578833,GLASS BON BON JAR,1,2011-11-25 15:23:00,3.29,NaN,United Kingdom
233083,558240,CLEAR DRAWER KNOB ACRYLIC EDWARDIAN,6,2011-06-27 15:27:00,2.46,NaN,United Kingdom
441855,575875,SET/4 SKULL BADGES,4,2011-11-11 13:06:00,0.83,NaN,United Kingdom
386929,571508,MEMO BOARD RETROSPOT DESIGN,1,2011-10-17 15:27:00,10.79,NaN,United Kingdom
477913,578347,WOODLAND CHARLOTTE BAG,1,2011-11-24 09:26:00,1.63,NaN,United Kingdom
77312,542998,JUMBO BAG TOYS,2,2011-02-02 12:16:00,4.13,NaN,United Kingdom
285972,562933,ROSES REGENCY TEACUP AND SAUCER,2,2011-08-10 16:51:00,5.79,NaN,United Kingdom


Kami tidak melihat suatu pattern atau alasan dari hilangnya customer ID. Menurut kami dengan hilangnya customerID tidak memengaruhi analisa kita terhadap data transaksi yang ada. Sehingga Kami tidak melakukan apa2 pada kolom CustomerID


## 1.2 Total Price

Total price digunakan untuk memudahkan pencatatan penjualan barang yang dibeli sesuai dengan quantity barang tersebut.

In [8]:
# Calculate the total price by multiplying the quantity and price columns
df['Total_Price'] = df.Quantity * df.Price

# Data Understanding: Exploring and Interpreting the Dataset

Exploring Overall sales
https://www.kaggle.com/code/thornr/pyspark-market-basket-analysis
-sales berdasarkan bulan & week
-dibawah ad 2 chart sales (top items)

Alasan knp pake 2011 only
https://www.kaggle.com/code/earije/market-basket-analysis-with-apriori

Exploring seasonal sensitive goods



# Association Rules

Association Rules dihasilkan menggunakan algoritma Apriori, dimana ia bertujuan untuk mencari asosiasi antar item yang sering dibeli bersama-sama.

Setiap rule terdiri dari dua bagian: the antecedent  and the consequent. Jika antecedent dibeli, maka ada suatu kemungkinan untuk consequent untuk dibeli juga. 


In [9]:
df2 = df

# Filter item name lebih dari sekali
item_counts = df2['Itemname'].value_counts(ascending=False)
filtered_items = item_counts.loc[item_counts > 1].reset_index()['index']
df2 = df2[df2['Itemname'].isin(filtered_items)]

# Filter billno lebih dari sekali
bill_counts = df2['BillNo'].value_counts(ascending=False)
filtered_bills = bill_counts.loc[bill_counts > 1].reset_index()['index']
df2 = df2[df2['BillNo'].isin(filtered_bills)]

Karena di analisis MBA membutuhkan minimal sebuah item untuk setidaknya dibeli lebih dari sekali (supaya bisa menjadi antecedent atau consequent), maka barang-barang yang hanya muncul sekali di dataframe dibuang untuk membuang data tidak yang tidak signifikan. Hal yang sama juga bisa dibilang untuk BillNo, jika barang yang dibeli di Bill yang sama, maka tidak bisa dibuat jadi sebuah pattern. 

Jadi df2 hanya akan contain rows dimana item name dan number yang terjadi lebih dari sekali.


In [ ]:
pivot_table = pd.pivot_table(df2[['BillNo','Itemname']], index='BillNo', columns='Itemname', aggfunc=lambda x: True, fill_value=False)

In [13]:
pivot_table.head()

Itemname,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,...,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
BillNo,,,,,,,,,,,,,,,,,,,,,
536365,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536366,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536367,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
536370,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Salah satu library yang telah mengimplementasi Apriori adalah mlxtend. Input function apriori mlxtend berupa sebuah tabel berisi True dan False dimana setiap row berupa sebuah bill dan kolom adalah item-itemnya. Cell akan diisi True jika di bill itu item dibeli dan False jika tidak.

In [10]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(pivot_table, min_support=0.01,use_colnames=True)
print("Frequent Itemsets:")
frequent_itemsets

Frequent Itemsets:


,support,itemsets
0,0.017372,(10 COLOUR SPACEBOY PEN)
1,0.013753,(12 MESSAGE CARDS WITH ENVELOPES)
2,0.019655,(12 PENCIL SMALL TUBE WOODLAND)
3,0.019822,(12 PENCILS SMALL TUBE RED RETROSPOT)
4,0.019599,(12 PENCILS SMALL TUBE SKULL)
...,...,...
2467,0.010356,"(LUNCH BAG CARS BLUE, LUNCH BAG PINK POLKADOT,..."
2468,0.010189,"(LUNCH BAG CARS BLUE, LUNCH BAG RED RETROSPOT,..."
2469,0.010301,"(LUNCH BAG CARS BLUE, LUNCH BAG WOODLAND, LUNC..."
2470,0.010468,"(LUNCH BAG PINK POLKADOT, LUNCH BAG WOODLAND, ..."


Pertama, kita perlu men-generate item set yang sering muncul bersamaan dalam suatu bill. Disini kami menggunakan minimum support 0.01%, yang berarti suatu itemset harus muncul setidaknya di 1% dati seluruh bill.

In [11]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, "confidence", min_threshold = 0.5)
print("\nAssociation Rules:")
rules


Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(60 CAKE CASES DOLLY GIRL DESIGN),(PACK OF 72 RETROSPOT CAKE CASES),0.023163,0.071214,0.013029,0.562500,7.898749,0.011379,2.122940,0.894108
1,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETROSPOT CAKE CASES),0.044432,0.071214,0.024220,0.545113,7.654594,0.021056,2.041794,0.909783
2,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE GREEN),0.023218,0.053563,0.015256,0.657074,12.267209,0.014012,2.759888,0.940314
3,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE PINK),0.023218,0.042261,0.011693,0.503597,11.916475,0.010711,1.929359,0.937858
4,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE RED),0.023218,0.057127,0.015813,0.681055,11.921784,0.014487,2.956226,0.937896
...,...,...,...,...,...,...,...,...,...,...
1392,"(CHARLOTTE BAG PINK POLKADOT, WOODLAND CHARLOT...","(CHARLOTTE BAG SUKI DESIGN, RED RETROSPOT CHAR...",0.014588,0.028118,0.011303,0.774809,27.555589,0.010893,4.315815,0.977976
1393,"(CHARLOTTE BAG PINK POLKADOT, RED RETROSPOT CH...","(CHARLOTTE BAG SUKI DESIGN, WOODLAND CHARLOTTE...",0.018096,0.025167,0.011303,0.624615,24.818788,0.010847,2.596891,0.977395
1394,"(WOODLAND CHARLOTTE BAG, STRAWBERRY CHARLOTTE ...","(CHARLOTTE BAG SUKI DESIGN, CHARLOTTE BAG PINK...",0.022494,0.018263,0.011303,0.502475,27.513584,0.010892,1.973243,0.985830
1395,"(CHARLOTTE BAG PINK POLKADOT, WOODLAND CHARLOT...","(CHARLOTTE BAG SUKI DESIGN, RED RETROSPOT CHAR...",0.021826,0.018486,0.011303,0.517857,28.014200,0.010899,2.035734,0.985821


Dari frequent itemsets yang sudah ada, kita dapat membuat association rules. Untuk menilai suatu rules, kita bisa menggunakan confidence, ia mengukur seberapa sering suatu item muncul di bill dengan item lain. Disini kita menggunakan minimum confidence 0.5 yang berarti hanya rules dengan confidence lebih dari dari 50% baru dimasukin.

In [12]:
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
17,(BEADED CRYSTAL HEART PINK ON STICK),(DOTCOM POSTAGE),0.011470,0.039310,0.011192,0.975728,24.821640,0.010741,39.580445,0.970848
614,"(HERB MARKER CHIVES, HERB MARKER THYME)",(HERB MARKER PARSLEY),0.010412,0.012918,0.010078,0.967914,74.929928,0.009943,30.764068,0.997035
607,"(HERB MARKER ROSEMARY, HERB MARKER CHIVES)",(HERB MARKER PARSLEY),0.010356,0.012918,0.010022,0.967742,74.916574,0.009888,30.599555,0.996977
619,"(HERB MARKER ROSEMARY, HERB MARKER CHIVES)",(HERB MARKER THYME),0.010356,0.012918,0.010022,0.967742,74.916574,0.009888,30.599555,0.996977
1219,"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY, HE...",(HERB MARKER THYME),0.010579,0.012918,0.010189,0.963158,74.561706,0.010053,26.792237,0.997137
...,...,...,...,...,...,...,...,...,...,...
25,(RED RETROSPOT CUP),(BLUE POLKADOT CUP),0.021381,0.018040,0.010690,0.500000,27.716049,0.010305,1.963920,0.984980
1159,"(RED RETROSPOT CHARLOTTE BAG, STRAWBERRY CHARL...","(CHARLOTTE BAG PINK POLKADOT, WOODLAND CHARLOT...",0.026837,0.021826,0.013419,0.500000,22.908163,0.012833,1.956347,0.982721
113,(HAND WARMER RED LOVE HEART),(HAND WARMER SCOTTY DOG DESIGN),0.021938,0.030234,0.010969,0.500000,16.537753,0.010306,1.939532,0.960606
147,(LOVE HOT WATER BOTTLE),(HOT WATER BOTTLE KEEP CALM),0.025835,0.042706,0.012918,0.500000,11.707953,0.011814,1.914588,0.938843


Disini kita bisa lihat top item sets yang paling sering muncul di dataset kita. Banyak item set yang antecedent dan consequent nya sejenis seperti Herb, Hand Warmer dan Cup.

In [12]:
rules.sort_values(by='support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
161,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.067241,0.114016,0.045551,0.677419,5.941463,0.037884,2.746552,0.891647
104,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.056174,0.054227,0.040656,0.723762,13.346921,0.037610,3.423766,0.980134
105,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.054227,0.056174,0.040656,0.749744,13.346921,0.037610,3.771438,0.978117
174,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.065517,0.114016,0.040100,0.612054,5.368164,0.032630,2.283785,0.870767
172,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.064794,0.114016,0.037597,0.580258,5.089283,0.030210,2.110781,0.859178
...,...,...,...,...,...,...,...,...,...,...
623,(HERB MARKER CHIVES),"(HERB MARKER THYME, HERB MARKER ROSEMARY)",0.011457,0.012069,0.010011,0.873786,72.399445,0.009873,7.827454,0.997618
611,(HERB MARKER CHIVES),"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY)",0.011457,0.011680,0.010011,0.873786,74.812760,0.009877,7.830538,0.998068
605,(HERB MARKER CHIVES),"(HERB MARKER MINT, HERB MARKER PARSLEY)",0.011457,0.011568,0.010011,0.873786,75.532114,0.009879,7.831420,0.998197
440,"(GREEN REGENCY TEACUP AND SAUCER, JUMBO STORAG...",(DOTCOM POSTAGE),0.011012,0.039377,0.010011,0.909091,23.086800,0.009577,10.566852,0.967338


Jika dilihat dari persenan seberapa sering consequent dibeli bareng antecedent, hal yang sama juga terjadi dimana kedua item dalam set sejenis / berhubungan

## Cross-Selling and Upselling

In [31]:
# Filter association rules for cross-selling opportunities
cross_selling_rules = rules[(rules['antecedents'].apply(len) == 1) & (rules['consequents'].apply(len) == 1)]

cross_selling_rules = cross_selling_rules.sort_values(by=['confidence', 'support'], ascending=False)

top_cross_selling = cross_selling_rules.head(5)

print("Cross-Selling Recommendations:")
for idx, row in top_cross_selling.iterrows():
    antecedent = list(row['antecedents'])[0]
    consequent = list(row['consequents'])[0]
    print(f"Customers who bought '{antecedent}' also bought '{consequent}'.")

Cross-Selling Recommendations:
Customers who bought 'BEADED CRYSTAL HEART PINK ON STICK' also bought 'DOTCOM POSTAGE'.
Customers who bought 'HERB MARKER THYME' also bought 'HERB MARKER ROSEMARY'.
Customers who bought 'HERB MARKER ROSEMARY' also bought 'HERB MARKER THYME'.
Customers who bought 'HERB MARKER CHIVES' also bought 'HERB MARKER PARSLEY'.
Customers who bought 'REGENCY TEA PLATE PINK' also bought 'REGENCY TEA PLATE GREEN'.


Dengan mengagmbil rules dimana sebuah produk dibeli sering dengan produk lain, kita dapat mengambil rekomendasi untuk melakukan cross-selling. 

In [32]:
# Filter association rules for upselling opportunities
upselling_rules = rules[(rules['antecedents'].apply(len) == 1) & (rules['consequents'].apply(len) > 1)]

upselling_rules = upselling_rules.sort_values(by=['confidence', 'support'], ascending=False)

top_upselling = upselling_rules.head(5)

top_upselling = upselling_rules.sort_values(['confidence', 'support'], ascending=False).drop_duplicates('antecedents')[:5]
for idx, row in top_upselling.iterrows():
    antecedent = list(row['antecedents'])[0]
    consequents = list(row['consequents'])
    print(f"For customers who bought '{antecedent}', recommend the following upgrades: {', '.join(consequents)}.")

For customers who bought 'HERB MARKER CHIVES', recommend the following upgrades: HERB MARKER PARSLEY, HERB MARKER THYME.
For customers who bought 'HERB MARKER THYME', recommend the following upgrades: HERB MARKER PARSLEY, HERB MARKER ROSEMARY.
For customers who bought 'HERB MARKER PARSLEY', recommend the following upgrades: HERB MARKER ROSEMARY, HERB MARKER THYME.
For customers who bought 'HERB MARKER ROSEMARY', recommend the following upgrades: HERB MARKER PARSLEY, HERB MARKER THYME.
For customers who bought 'REGENCY TEA PLATE PINK', recommend the following upgrades: REGENCY TEA PLATE ROSES, REGENCY TEA PLATE GREEN.


In [ ]:
rules['antecedent_item'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequent_item'] = rules['consequents'].apply(lambda x: list(x)[0])

In [ ]:
# Merge with item prices to get prices of antecedent and consequent items

rules = rules.merge(df2, left_on='antecedent_item', right_on='Itemname', how='left').rename(columns={'Price': 'antecedent_price'}).drop(columns=['Itemname'])
rules = rules.merge(df2, left_on='consequent_item', right_on='Itemname', how='left').rename(columns={'Price': 'consequent_price'}).drop(columns=['Itemname'])
# Filter rules where the consequent item is higher-priced than the antecedent item
upselling_rules = rules[rules['consequent_price'] > rules['antecedent_price']]

# Sort by confidence and support in descending order
upselling_rules = upselling_rules.sort_values(by=['confidence', 'support'], ascending=False)

# Select the top 5 upselling rules
top_upselling = upselling_rules.head(5)

# Display the top upselling rules
print(top_upselling)

Selain cross-selling, kita juga dapat mengambil rekomendasi untuk upselling dengan berfokus kepada rules dimana terjadi priduct upgrades atau higher priced alternatives. 

# Conclusion

In this project, we explored the concept of association rules using the Apriori algorithm and the mlxtend library in Python. Association rules analysis provides valuable insights into the relationships and patterns within a dataset, enabling businesses to uncover hidden associations between items and make informed decisions for various applications.

We started by preparing the data and filtering out infrequent items and irrelevant transactions. Then, we generated frequent itemsets and association rules based on predefined thresholds for support and confidence. These rules allowed us to identify significant associations between items and quantify their strength.

The generated association rules provided actionable insights for different business scenarios. We explored cross-selling opportunities by identifying products frequently purchased together. By leveraging these associations, businesses can implement effective cross-selling strategies, offering relevant add-on products or upgrades to customers, thereby increasing revenue.

Additionally, we examined upselling recommendations, focusing on identifying suitable product upgrades or higher-priced alternatives for customers. By considering only one product recommendation for each top item, we ensured diverse and relevant suggestions, avoiding repetitive recommendations and enhancing the upselling strategy.

Furthermore, we discussed the importance of interpreting the support, confidence, lift, leverage, and conviction metrics associated with association rules. These metrics provide quantitative measures of the strength, significance, and impact of the associations, enabling businesses to prioritize and optimize their decision-making processes.

Overall, association rules analysis offers valuable insights and practical applications across various domains, such as marketing, product recommendations, cross-selling strategies, and process optimization. By understanding the associations between items, businesses can make data-driven decisions, improve customer satisfaction, enhance marketing campaigns, and drive business growth.

It is important to note that the analysis and insights provided in this project are specific to the dataset and parameters used. The results can be further refined and customized based on the specific requirements, domain knowledge, and business objectives.
